## Initial Scrape of NTEE codes and mission statements from guidestar API

In [287]:
from time import sleep 
import re
from tqdm import tqdm # track progress of loop
import http.client, urllib.request, urllib.parse, urllib.error, base64

In [410]:
# intialise request body in JSON format as Str
body = '''{
from: 0,
  size: 25,
    filters: {
      geography: {
        state: ['ID', 'AK', 'WA', 'OR', 'MO'],
    }
  }
}'''

In [411]:
# initialize other components
headers = {
    # Request headers
    'Content-Type': 'application/json',
    'Subscription-Key': '52d3100fdccb41a9991c95814cbd0b5d',
}

params = urllib.parse.urlencode({
})

    

In [415]:
container =[]

In [404]:

start = ['0']+[str(x) for x in range(26, 9977, 25)]
end = [str(x) for x in range (25, 10001, 25)]

In [405]:
x =0 # track num of calls

for num1, num2 in tqdm(zip(start, end)):
    try:
        conn = http.client.HTTPSConnection('apidata.guidestar.org')
        body = re.sub(num1, str(int(num1)+25), body) #increment start pagination of results
        body = re.sub(num2, str(int(num2)+25), body) #increment end pagination of result
        conn.request("POST", "/essentials/v1?%s" % params, body, headers)
        response = conn.getresponse()
        data = response.read()
        container.append(str(data).split(','))
        conn.close()
        x+=1
        if x%10 ==0: #delay calls for 50 seconds every 10  iteration to avoid 429 status errors
            sleep(50)
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))
        x+=1
        if x%10 ==0:
            sleep(50)

74it [09:46, 37.16s/it]

[Errno 8] nodename nor servname provided, or not known
[Errno 8] nodename nor servname provided, or not known
[Errno 8] nodename nor servname provided, or not known
[Errno 8] nodename nor servname provided, or not known
[Errno 8] nodename nor servname provided, or not known
[Errno 8] nodename nor servname provided, or not known
[Errno 8] nodename nor servname provided, or not known


79it [09:47, 26.02s/it]

[Errno 8] nodename nor servname provided, or not known
[Errno 8] nodename nor servname provided, or not known
[Errno 8] nodename nor servname provided, or not known


400it [40:54, 15.92s/it]


In [377]:
#strip extra double quotes from extracted data
for i, line in enumerate(filtered_data):
    filtered_data[i] = line.replace('\"', '')

In [385]:
# filter data field of interest
data = {'ein':[],'organization_name':[],'mission':[],'ntee_code':[]}
for line in filtered_data:
        if line.startswith('ein'):
            data['ein'].append(line.split(':')[1])
        elif line.startswith('organization_name'):
            data['organization_name'].append(line.split(':')[1])
        elif line.startswith('mission'):
            data['mission'].append(line.split(':')[1])
        elif line.startswith('ntee_code'):
            data['ntee_code'].append(line.split(':')[1])

In [1]:
import pandas as pd
import numpy as np

In [389]:
df = pd.DataFrame.from_dict(data)

In [439]:
print('info extracted from {} non-prfits'.format(df.shape[0]))
df.head()

info extracted from 4475 non-prfits


,ein,organization_name,mission,ntee_code
0,46-2657636,Wenatchee Christian Early Learning,To Provide Childcare For Children Ages 1-12,P33 Child Day Care
1,46-2667218,Katy Land Trust Inc,The Katy Land Trust Seeks To Educate All Misso...,C34 Land Resources Conservation
2,46-2675536,Biggest Show in Idaho Music Festival & Extrava...,Community Music Festival And Fireworks Event,A68 Music
3,47-0915970,Ozark Adventures Inc,To Provide And Promote The History And Culture...,B60 Adult
4,46-4626729,Friends of Wyatt,NaN,Z99 Unknown


In [440]:
df['mission'] = df.mission.replace("", np.NaN)

df2 =df.dropna()
print('mission statements extracted from {} non-prfits'.format(df2.shape[0]))
df2.head()


mission statements extracted from 3448 non-prfits


,ein,organization_name,mission,ntee_code
0,46-2657636,Wenatchee Christian Early Learning,To Provide Childcare For Children Ages 1-12,P33 Child Day Care
1,46-2667218,Katy Land Trust Inc,The Katy Land Trust Seeks To Educate All Misso...,C34 Land Resources Conservation
2,46-2675536,Biggest Show in Idaho Music Festival & Extrava...,Community Music Festival And Fireworks Event,A68 Music
3,47-0915970,Ozark Adventures Inc,To Provide And Promote The History And Culture...,B60 Adult
5,46-2788508,West Salem Band Boosters,To Provide Finanacial And Volunteer Support Fo...,B11 Single Organization Support


In [434]:
# save dataframe to file for later merge with Irs masterfile
df.to_csv('/Users/markespina/src/DataScienceProjects/data/fundamentals/guidestar_sample.csv')